# Training and Testing Dataset Overview

# Introduction
In the realm of environmental research and forecasting, access to reliable and comprehensive datasets is paramount. Four such indispensable sources are gridMET, AMSR, MODIS, and SNOTEL. Each dataset offers unique insights into various aspects of Earth's climate and terrain, contributing significantly to our understanding of environmental dynamics. These datasets serve as the cornerstone for numerous applications, including weather forecasting, ecological modeling, and climate change research. In our **snowcast_wormhole** workflow, we leverage the richness of these datasets to enhance the accuracy and robustness. Let's embark on a detailed exploration of these invaluable data sources.


# gridMET Dataset

## Introduction
gridMET is like having a weather wizard at your fingertips! It's a dataset packed with daily high-spatial resolution (~4-km, 1/24th degree) surface meteorological data covering the contiguous US, spanning from 1979 up to yesterday. It even extends its reach to cover southern British Columbia in real-time products. This dataset provides vital insights into weather patterns, aiding various industries and scientific endeavors.

## Insights

* It offers a fascinating glimpse into historical weather patterns, allowing users to delve as far back as 1979.
* The dataset provides an extensive array of information, including maximum and minimum temperatures, precipitation, radiation, wind velocity, and various other meteorological variables.
* Scientists and modelers find it invaluable for a myriad of purposes, such as ecological research, agricultural strategizing, hydrological modeling, and beyond.

## Characteristics

- **Spatial Resolution**: GridMET provides data at a high spatial resolution, typically around 4 kilometers.
  
- **Temporal Resolution**: The dataset offers data at a daily time step, allowing access to meteorological data for each day.
  
- **Variables**: GridMET includes meteorological variables such as temperature, precipitation, humidity, wind speed, and solar radiation.
  
- **Coverage**: The dataset covers the contiguous United States, providing comprehensive meteorological data for this region.
  
- **Quality**: Data is derived from multiple sources including weather station observations, satellite data, and meteorological models, and is quality-controlled.
  
- **Long-term Availability**: GridMET provides data spanning several decades, enabling analysis of historical meteorological trends and patterns.

## Data Format:

GRIDMET data is typically available in common file formats such as NetCDF (Network Common Data Form) or ASCII (plain text) format. NetCDF is widely used for multidimensional scientific data and provides efficient storage and access to large datasets. ASCII format offers simplicity and compatibility with various software tools for data analysis and visualization.

## Data Sources and Acquisition:

#### Satellite Data:
GRIDMET utilizes satellite data from different sources such as the Geostationary Operational Environmental Satellite (GOES) and the Moderate Resolution Imaging Spectroradiometer (MODIS). These satellite observations provide valuable information on temperature, precipitation, humidity, and other atmospheric variables.

#### Ground-Based Observations:
The dataset incorporates ground-based weather station observations from various networks such as the National Weather Service's Cooperative Observer Program (COOP), the Automated Surface Observing System (ASOS), and the Automated Weather Data Network (AWDN). These observations help in validating and enhancing the accuracy of the dataset.

#### Meteorological Models:
gridMET also integrates data from numerical weather prediction models like the North American Mesoscale Forecast System (NAM) and the Weather Research and Forecasting (WRF) model. These models provide forecasts and simulations of meteorological variables which are then blended with observed data to create a more complete dataset.

## Applications:

- Climate research and modeling
- Agriculture and crop management
- Hydrology and water resource management
- Renewable energy assessment
- Disaster management and risk assessment
- Ecological and environmental studies

# gridMET Data Download Instructions

There are several options available for acquiring gridMET data. Below are the methods you can use to obtain the dataset:


1. **Direct Download of NetCDF Files:**
   - Download NetCDF files directly from the [NetCDF data repository](https://www.northwestknowledge.net/metdata/data/).

2. **Create wget Script for Downloading NetCDF Files:**
   - Script to automate downloading of NetCDF files using [wget](https://www.climatologylab.org/wget-gridmet.html).

4. **THREDDS Catalog (OPENDAP):**
   - Access data via [THREDDS Catalog](http://thredds.northwestknowledge.net:8080/thredds/reacch_climate_MET_catalog.html) with OPENDAP protocol.

5. **Aggregated THREDDS Catalog (OPENDAP):**
   - Access aggregated data via [Aggregated THREDDS Catalog](http://thredds.northwestknowledge.net:8080/thredds/reacch_climate_MET_aggregated_catalog.html) with OPENDAP protocol.


6. **Elevation Data on the Same 4-km Grid:**
   - Access elevation data on the same grid as the meteorological data from [elevation_dataset](https://climate.northwestknowledge.net/METDATA/data/metdata_elevationdata.nc).

7. **Data Subsets using Web Tools:**
   - [ClimateEngine.org](http://climateengine.org)
   - [Geo Data Portal (GDP)](https://labs.waterdata.usgs.gov/gdp_web/client/#!catalog/gdp/dataset/54dd5df2e4b08de9379b38d8) ([Tutorial on GDP Tool](https://climate.northwestknowledge.net/MACA/GDP.php))


For more information on the gridMET dataset, you can visit the [gridMET website](https://www.climatologylab.org/gridmet.html). Here, you'll find detailed documentation.

# AMSR Dataset

## Introduction
### What is AMSR?
AMSR, short for Advanced Microwave Scanning Radiometer is a satellite-based instrument designed to measure microwave emissions from the Earth's surface and atmosphere. The AMSR dataset encompasses satellite-derived microwave observations crucial for a myriad of applications including weather forecasting, climate monitoring, and environmental studies. These datasets provide valuable insights into geophysical parameters such as sea surface temperature, soil moisture, sea ice concentration, precipitation, and wind speed over oceans. By collecting data on these factors, AMSR helps scientists better understand how water moves and behaves across the planet. Its observations provide valuable insights into Earth's climate dynamics and hydrological processes, contributing to our overall understanding of the environment.

### Quick Look at the Data Collections
AMSR-E, or the Advanced Microwave Scanning Radiometer for the Earth Observing System, is like a high-tech detective tool that travels on NASA's Aqua satellite. It gathers information about water on Earth and other important details about our environment. AMSR-E data collections include AMSR/ADEOS-II and AMSR2, hosted by the National Snow and Ice Data Center Distributed Active Archive Center (NSIDC DAAC). These datasets provide a treasure trove of polar observations, facilitating research in climatology, oceanography, hydrology, and more.

## Characteristics
- **Spatial Resolution**: AMSR provides data at various spatial resolutions depending on the specific instrument version and product. Typically, it offers moderate to high spatial resolution microwave data.

- **Temporal Resolution**: AMSR data is available at various temporal resolutions, ranging from daily to monthly, depending on the specific product and parameters.

- **Variables**: AMSR measures various parameters such as soil moisture, sea surface temperature, precipitation, snow water equivalent, and sea ice concentration using microwave radiometry.

- **Coverage**: AMSR provides global coverage, allowing for the monitoring of environmental parameters across different regions of the Earth.

- **Quality**: Data from AMSR undergoes rigorous quality control processes to ensure accuracy and reliability, considering factors such as calibration and validation against ground-based measurements.

## Data Format

The Advanced Microwave Scanning Radiometer (AMSR) data is commonly stored in HDF5 format, providing a hierarchical structure for organizing datasets such as brightness temperature, sea surface temperature, and soil moisture. Within the HDF5 file, data parameters are organized into groups, with subgroups representing different configurations or subsets of the data. Metadata and geolocation information are typically stored separately, offering additional details about the data.

## AMSR Data Collections
### AMSR/ADEOS-II
AMS/ADEOS-II, operating on the Advanced Earth Observing Satellite-II (ADEOS-II) platform, was the initial version of the AMSR instrument, providing passive microwave measurements from early 2003 to October 24, 2003. Offering Level-1A and Level-2A products, it served as a crucial tool for studying changes in polar ice caps, global precipitation patterns, and oceanic circulation. This data has been instrumental in enhancing our understanding of Earth's climate dynamics and informing research across various scientific disciplines.

### AMSR2
AMSR2, launched in 2012 onboard the Global Change Observation Mission-Water (GCOM-W1) satellite, represents the next generation of AMSR instruments. It continues the legacy of AMSR-E, capturing observations with improved spatial resolution and enhanced capabilities. AMSR2 data has been instrumental in monitoring sea ice extent, ocean surface winds, and soil moisture dynamics at global scales.

### Hosted by NSIDC DAAC
The NSIDC DAAC serves as the primary repository for AMSR-related data, ensuring that these valuable datasets are freely accessible to the scientific community and the public. The center provides data discovery, access, and user support services, facilitating research in cryospheric and hydrological sciences, climate modeling, and environmental monitoring.

## Applications
1. **Weather Forecasting**: AMSR data aids in short-term weather forecasting by providing insights into atmospheric moisture content and precipitation patterns.

2. **Climate Monitoring**: AMSR datasets contribute to long-term climate monitoring efforts by providing data on parameters like sea surface temperature and sea ice concentration.

3. **Oceanography**: AMSR data enables the study of ocean surface properties such as sea surface temperature, wind speed, and ocean salinity, facilitating research in oceanography and marine science.

4. **Hydrology**: AMSR-derived soil moisture data supports hydrological modeling, drought monitoring, and water resource management by providing information on soil moisture content and surface water availability.

5. **Cryosphere Studies**: AMSR datasets are crucial for studying the cryosphere, including monitoring changes in snow cover extent, snow water equivalent, and sea ice concentration, contributing to our understanding of climate change impacts in polar regions.

# AMSR Dataset Download Instructions

### Step 1: Access the Data Portals
- For Standard Products, visit the [Globe Portal System (G-Portal)](https://gportal.jaxa.jp/gpr/) and sign in or create an account.
- For Research Products, access the [GCOM-W Research Product Distribution Service](https://suzaku.eorc.jaxa.jp/GCOM_W/research/resdist.html) and log in or register.

### Step 2: Select the Desired Product
- Choose the appropriate AMSR product based on your requirements (AMSR2, AMSR-E, or AMSR-E in slow rotation mode).

### Step 3: Define Data Range
- Specify the date range for the data you wish to download.

### Step 4: Initiate Download
- Follow the prompts to start the download process after specifying the product and date range.

Remember to adhere to any terms of use or licensing agreements associated with the data you're downloading.


For more information, visit the [AMSR-E Sensor Page](https://www.earthdata.nasa.gov/sensors/amsr-e).

# MODIS Dataset

## Introduction
### What is MODIS?
MODIS, which stands for MODerate Resolution Imaging Spectroradiometer, is an advanced instrument operating aboard both the Terra and Aqua spacecraft. It captures a comprehensive view of Earth's surface, oceans, and atmosphere. The MODIS dataset is a comprehensive collection of Earth observation data captured by the MODIS instruments. Scientists use MODIS data to track changes in things like land cover, weather patterns, ice and snow, and the color of the oceans. MODIS boasts a remarkable viewing swath width of 2,330 km and covers the entire Earth surface every one to two days. With 36 spectral bands ranging from 0.405 to 14.385 µm, it provides detailed data at three spatial resolutions: 250m, 500m, and 1,000m.

## Characteristics
- **Spatial Resolution**: MODIS provides data at moderate spatial resolutions ranging from 250 meters to 1 kilometer, depending on the specific product and spectral band.

- **Temporal Resolution**: MODIS offers daily global coverage, providing data at a high temporal resolution suitable for monitoring dynamic environmental processes.

- **Variables**: MODIS measures various Earth surface parameters including land cover, land surface temperature, vegetation indices, fire occurrence, ocean color, and atmospheric properties.

- **Coverage**: MODIS provides global coverage, capturing data over land, ocean, and atmosphere, facilitating multi-disciplinary Earth observation studies.

- **Quality**: MODIS data undergoes extensive calibration and validation processes to ensure accuracy and reliability, with quality flags provided to identify potential data anomalies.

## Data Format
MODIS data are typically available in Hierarchical Data Format (HDF) or NetCDF formats, which are widely used for storing and distributing Earth observation data. These formats facilitate efficient data access, manipulation, and analysis using various software tools and programming languages commonly employed in the Earth sciences community.

## MODIS Direct Broadcast
Users with x-band receiving systems can capture regional data directly from the spacecraft using the MODIS Direct Broadcast signal, enhancing real-time monitoring capabilities.

# MODIS Dataset Download Instructions

### Step 1: Define Your Study Area (AOI)
- Define your area of interest (AOI) by either specifying the x,y vertices of each corner or by uploading a zipped shapefile containing the boundary.
- Ensure your extent polygon is square or rectangular with minimal vertices.
- Visit the [Earth Explorer website](https://earthexplorer.usgs.gov/) and log in or create an account if you haven't already.

### Step 2: Define the Data That You Want to Download
- Navigate to the **Data sets** tab
- Choose the **MODIS Land Surface Reflectance - V6** dataset from the NASA LPDAAC Collections dropdown.
- Select the checkbox for **MODIS MOD09GA V6**.

### Step 3: Define Selection Criteria
- Click on the **Additional Criteria** tab.
- Set filters as needed, such as limiting results to daytime images.

### Step 4: View Results & Select Data to Order / Download
- Click on the **Results** tab to view available scenes covering your study area.
- Use icons below the scene to visualize it on the map and see its footprint relative to your study area.

### Step 5: Download Your Data
- Click the download data icon.
- Choose the **HDF Format** option in the pop-up window.
- Log in to your Earthdata account through [NASA](https://urs.earthdata.nasa.gov/).
- Follow prompts to confirm and start the download.

*Remember*, if you don't have an Earthdata account, you'll need to create one to access the data. Ensure you provide an email and adjust communication preferences accordingly.

Once logged in, the download will begin, and you can start using your MODIS data for your intended purposes.

For more information, visit the [MODIS Sensor Page](https://www.earthdata.nasa.gov/sensors/modis).

# SNOTEL dataset

## Introduction
### What is SNOTEL?
The SNOwpackTELemetryNetwork (SNOTEL) is an automated system of snowpack and climate sensors managed by the Natural Resources Conservation Service (NRCS) in the Western United States, offering critical data for water supply forecasting, flood prediction, and climate research. SNOTEL provides real-time data on snow water equivalent, snow depth, precipitation, and temperature from remote mountainous regions, aiding in understanding hydroclimatic conditions. SNOTEL offers comprehensive snowpack and climate data from over 900 sites, helping monitor snowpack, precipitation, temperature, and other climatic conditions in the western U.S.The SNOTEL dataset serves as a valuable resource for a wide range of stakeholders, contributing to informed decision-making in various sectors impacted by snowpack and climate conditions.

## SNOTEL Network Overview
### Composition of SNOTEL
- Comprising over 900 automated sites in remote, high-elevation mountain watersheds.
- Monitors snowpack, precipitation, temperature, and other climatic parameters.

### Operations and Data Collection
- Sites operate unattended and without maintenance for extended periods.
- Standard sensor configuration includes snow pillow, precipitation gauge, and temperature sensors.

## Telemetry and Data Transmission
### Data Collection and Storage
- Dataloggers installed in equipment shelters collect and store data.
- Various telemetry systems transmit data back to the Water and Climate Information System.

### Enhanced Site Capabilities
- Enhanced sites equipped with soil moisture, soil temperature, solar radiation, wind speed, and relative humidity sensors.
- Tailored configurations based on physical conditions and climate requirements.

## Characteristics
**Spatial Resolution**: SNOTEL provides data at a network of monitoring sites distributed across mountainous regions, typically covering areas with varying spatial resolutions depending on the density of monitoring stations.

**Temporal Resolution**: SNOTEL data is typically collected at hourly intervals, providing high temporal resolution data for monitoring snowpack conditions and related hydrological variables.

**Variables**: SNOTEL measures snow water equivalent, snow depth, temperature, precipitation, and soil moisture at monitoring sites in mountainous regions.

**Coverage**: SNOTEL stations are primarily located in the western United States, covering areas with significant snowpack and water resource management importance.

**Quality**: SNOTEL data undergoes quality control procedures to ensure accuracy and reliability, including calibration checks and validation against manual measurements.

## Data Format
The Snow Telemetry (SNOTEL) data format encompasses structured datasets collected from remote automated stations situated in mountainous regions, monitoring snowpack, weather, and hydrological parameters. Key aspects include recorded parameters such as snow water equivalent (SWE), snow depth, air temperature, and precipitation, timestamped to denote observation times and often stored at varying resolutions like hourly or daily intervals. Quality control flags accompany data points to denote reliability, while metadata provides station details and sensor calibration information. SNOTEL data is commonly stored in formats like CSV, TSV, HDF5, or netCDF, accessible through agency websites, data portals, or APIs. This format facilitates applications spanning water resource management, climate research, agriculture, recreation, hydrological modeling, and ecological studies.


## Applications

**Water Resource Management**:
   - *Snowpack Monitoring*: Assessing snowpack depth and SWE helps in forecasting water availability for irrigation, hydropower generation, and municipal water supply.
   - *Runoff Forecasting*: Data from SNOTEL stations aids in predicting spring runoff, facilitating reservoir management and flood control.

**Climate Research**:
   - *Long-term Climate Trends*: Historical data enables researchers to study long-term climate patterns, including changes in snowfall, temperature, and precipitation.
   - *Climate Change Studies*: SNOTEL data is utilized to understand the impacts of climate change on snowpack dynamics, water resources, and ecosystems.

**Agriculture and Forestry**:
   - *Crop Planning*: Farmers use snowpack data to anticipate water availability during the growing season, aiding in crop planning and irrigation scheduling.
   - *Forest Management*: Forestry agencies utilize SNOTEL data for assessing wildfire risk, planning timber harvests, and monitoring forest health.

**Recreation and Tourism**:
   - *Winter Sports Planning*: Ski resorts and recreational outfitters rely on snowpack data for planning activities such as skiing, snowboarding, and snowmobiling.
   - *Summer Recreation*: Understanding snowmelt timing and water availability helps in planning summer recreational activities like hiking, fishing, and camping.

# SNOTEL Dataset Download Instructions

Because snow has a higher albedo than most other land cover types, it can cause the seasonal changes in the albedo of a landscape to be quite dramatic. The Soil Climate Analysis Network (SCAN) and the SNOwpack TELemetry (SNOWTEL) network provide snow depth and snow water equivalent (the amount of water contained in a snowpack) data for many sites across the United States.

### Step 1: Navigate to the SCAN/SNOWTEL website
- Visit the [SCAN/SNOWTEL Website](http://www.wcc.nrcs.usda.gov/nwcc/inventory).

### Step 2: Choose Data Product and Location
- Select the data product you are interested in (e.g., Snow Depth or Snow Water Equivalent) from the drop-down menu under *Element*.
- Choose a State/County or Basin using the drop-down menus provided.

### Step 3: View Inventory
- Click on 'View Inventory' to see available stations in your selected area.
  - If no results are returned, consider widening your search.

### Step 4: Select Station and Data
- Click 'View' next to the station of interest to access its page.
- Use the table to select the data you need:
  - Choose the data product (Snow Depth or Snow Water Equivalent).
  - Select ‘Daily’ in the Time Series column.
  - Choose the format ('chart' for visualization or 'csv' for download).
  - View current data by selecting the desired time frame in the yellow column and clicking 'View Current', or view historic data by selecting the year and time in the green column and clicking 'View Historic'.

### Step 5: Download Data
- Save the downloaded CSV file to your computer for further analysis.

**For map visualization of SNOWTEL stations, click on ‘SNOWTEL data’ under ‘Climate Monitoring’ in the right panel. The maps are clickable for station selection.**

For more information, visit the [NRCS SNOTEL page](https://www.nrcs.usda.gov/wps/portal/wcc/home/aboutUs/monitoringPrograms/automatedSnowMonitoring).
